In [17]:
import pandas as pd

# Use raw string to correctly format the file path on Windows
url = r'C:\Users\X521\OneDrive - Universidade de Lisboa\Ambiente de Trabalho\tecnico\Master\Advanced Automation\cenas\AutoAvancada\data\raw\outputRoary24_90\gene_presence_absence.csv'

# Read the CSV file from the local path
df = pd.read_csv(url)

# Function to get the annotation or function of a gene
def get_gene_function(data, gene_name):
    """
    Retrieve the function or annotation for a specified gene.

    Parameters:
    - data: pandas DataFrame containing the data.
    - gene_name: The name of the gene to search for.

    Returns:
    - A string with the gene's function or annotation, or an error message.
    """
    # Check if necessary columns exist
    if "Gene" not in data.columns or "Annotation" not in data.columns:
        return "The necessary columns ('Gene' and 'Annotation') are missing in the data."
    
    # Search for the gene in the 'Gene' column
    result = data[data['Gene'] == gene_name]
    
    if result.empty:
        return f"Gene '{gene_name}' not found in the dataset."
    
    # Return the annotation (function) of the gene
    return result['Annotation'].iloc[0]

# Example usage
gene_name = "rpmG"  # Replace this with the gene you're searching for
function = get_gene_function(df, gene_name)
print(function)



50S ribosomal protein L33
